#### Choose a plain image P and a secret key K of 32 bytes.

In [16]:
import numpy as np
from PIL import Image
import os, shutil

In [27]:
# Rename files in test_images folder to aa, ab, ac, etc.
# This is necessary for the code to work properly.

# Path to test_images folder
path = 'test_images/'

# List of files in test_images folder
files = os.listdir(path)

# Rename files in test_images folder to aa, ab, ac, etc.
for i, file in enumerate(files):
    os.rename(os.path.join(path, file), os.path.join(path, 'a' + str(chr(97 + i)) + '.jpg'))

In [28]:
np.random.seed(42)

def generate_latin_square(size):
    latin_square = np.arange(size)
    np.random.shuffle(latin_square)
    return latin_square

# Step 2: Latin Square Whitening
def latin_square_whitening(data, latin_squares):
    # Get the dimensions of the image
    height, width, channels = data.shape
    whitened_data = np.zeros_like(data, dtype=np.uint8)
    for channel in range(channels):
        whitened_data[:, :, channel] = data[:, :, channel] ^ latin_squares[channel][data[:, :, channel]]
    return whitened_data

# Step 3: Latin Square Substitution
def latin_square_substitution(data, latin_square, key):
    # Apply Latin square substitution
    substituted_data = np.array([latin_square[byte] for byte in data], dtype=np.uint8)
    
    # XOR with the encryption key for each element
    key = np.frombuffer(key, dtype=np.uint8)  # Convert the key to the same data type
    key = np.tile(key, len(substituted_data) // len(key) + 1)[:len(substituted_data)]  # Reshape the key
    substituted_data = substituted_data ^ key
    
    return substituted_data


# Step 4: Latin Square Permutation
def latin_square_permutation(data, latin_square, key):
    # Get the size of the data
    size = len(data)
    
    # Apply Latin square permutation
    permuted_data = np.zeros(size, dtype=np.uint8)
    for i, byte in enumerate(data):
        bit_position = latin_square[i]
        permuted_data[bit_position] = byte
    
    # XOR with the encryption key for each element
    key = np.frombuffer(key, dtype=np.uint8)  # Convert the key to the same data type
    key = np.tile(key, size // len(key) + 1)[:size]  # Reshape the key
    permuted_data = permuted_data ^ key
    
    return permuted_data

def reverse_latin_square_permutation(data, latin_square, key):
    size = len(data)
    reversed_permutation = np.zeros(size, dtype=np.uint8)
    for i in range(size):
        reversed_permutation[latin_square[i]] = data[i]
    key = np.frombuffer(key, dtype=np.uint8)  
    key = np.tile(key, size // len(key) + 1)[:size]  
    reversed_permutation = reversed_permutation ^ key
    return reversed_permutation

def reverse_latin_square_substitution(data, latin_square, key):
    reverse_latin_square = np.argsort(latin_square)
    reversed_substitution = np.array([reverse_latin_square[byte] for byte in data], dtype=np.uint8)
    key = np.frombuffer(key, dtype=np.uint8)  
    key = np.tile(key, len(reversed_substitution) // len(key) + 1)[:len(reversed_substitution)]  
    reversed_substitution = reversed_substitution ^ key
    return reversed_substitution

In [32]:
# Main program
# if __name__ == "__main__":
counter = 0
counter2 = 0

shutil.rmtree("encrypted_images", ignore_errors=True)
os.mkdir("encrypted_images")

shutil.rmtree("decrypted_images", ignore_errors=True)
os.mkdir("decrypted_images")


for filename in os.listdir("test_images"):
    # Encryption --------------------------------------------
    src = "test_images"
    dest = "encrypted_images"

    # Load an example image (replace with your image loading code)
    image_path = os.path.join(src, filename)  # Replace with the path to your image file
    image = Image.open(image_path)

    # Convert the image to a numpy array
    image_data = np.array(image)
    

    # Step 1: Generate a secret key K (32 bytes)
    secret_key = np.random.bytes(64)

    # Generate Latin squares for each channel
    channels = image_data.shape[-1]
    latin_squares = [generate_latin_square(512) for _ in range(channels)]

    # Generate a Latin square (substitution matrix) for the given size
    latin_square = generate_latin_square(image_data.size)

    # Step 2: Latin Square Whitening
    whitened_data = latin_square_whitening(image_data, latin_squares)

    # Step 3: Latin Square Substitution
    substituted_data = latin_square_substitution(whitened_data.flatten(), latin_square, secret_key)

    # Step 4: Latin Square Permutation
    permuted_data = latin_square_permutation(substituted_data, latin_square, secret_key)
    permuted_data_shape = permuted_data.shape

    # After completing all encryption rounds, you can obtain the encrypted image
    encrypted_image = permuted_data.reshape(image_data.shape)

    # Display the encrypted image
    encrypted_image = Image.fromarray(encrypted_image)
    # encrypted_image.show()  # Show the image in a default image viewer

    # Alternatively, you can save the encrypted image to a file
    encrypted_image.save(os.path.join(dest, f"EncryptedImage_a{str(chr(97 + counter))}.png"))
    counter += 1

    # Decryption --------------------------------------------
    src = "encrypted_images"
    dest = "decrypted_images"

    # Reverse Step 4: Latin Square Permutation
    unpermuted_data = reverse_latin_square_permutation(image_data.flatten(), latin_square, secret_key)

    # Reverse Step 3: Latin Square Substitution
    unsubstituted_data = reverse_latin_square_substitution(unpermuted_data, latin_square, secret_key)

    # Reshape data back to original dimensions
    unsubstituted_data = unsubstituted_data.reshape(image_data.shape)

    # Reverse Step 2: Latin Square Whitening
    unwhitened_data = latin_square_whitening(unsubstituted_data, latin_squares)

    decrypted_image = Image.fromarray(unwhitened_data)
    
    decrypted_image.save(os.path.join(dest, f"DecryptedImage_a{str(chr(97 + counter2))}.png"))
    counter2 += 1

    print(f"Encrypted and decrypted {filename} successfully!")

Encrypted and decrypted aa.jpg successfully!
Encrypted and decrypted ab.jpg successfully!
Encrypted and decrypted ac.jpg successfully!
Encrypted and decrypted ad.jpg successfully!
Encrypted and decrypted ae.jpg successfully!
Encrypted and decrypted af.jpg successfully!
Encrypted and decrypted ag.jpg successfully!
Encrypted and decrypted ah.jpg successfully!
Encrypted and decrypted ai.jpg successfully!
Encrypted and decrypted aj.jpg successfully!
Encrypted and decrypted ak.jpg successfully!
Encrypted and decrypted al.jpg successfully!
Encrypted and decrypted am.jpg successfully!
Encrypted and decrypted an.jpg successfully!
Encrypted and decrypted ao.jpg successfully!
Encrypted and decrypted ap.jpg successfully!
Encrypted and decrypted aq.jpg successfully!
Encrypted and decrypted ar.jpg successfully!
Encrypted and decrypted as.jpg successfully!
Encrypted and decrypted at.jpg successfully!


In [38]:
# Encryption --------------------------------------------
src = "test_images"
dest = "encrypted_images"

counter = 0
counter2 = 0

shutil.rmtree("encrypted_images", ignore_errors=True)
os.mkdir("encrypted_images")

shutil.rmtree("decrypted_images", ignore_errors=True)
os.mkdir("decrypted_images")

# Load an example image (replace with your image loading code)
image_path = os.path.join(src, 'aa.jpg')  # Replace with the path to your image file
image = Image.open(image_path)

# Convert the image to a numpy array
image_data = np.array(image)
print(f"\nImage shape: \n{image_data.shape} \n\nImage Data: \n{image_data}")

# Step 1: Generate a secret key K (32 bytes)
secret_key = np.random.bytes(32)
print(f"\nSecret key: \n{secret_key}")

# Generate Latin squares for each channel
channels = image_data.shape[-1]
latin_squares = [generate_latin_square(512) for _ in range(channels)]
print(f"\nChannels: \n{channels} \n\nLatin Squares: ")
for i in latin_squares:
    print(i)

# Generate a Latin square (substitution matrix) for the given size
latin_square = generate_latin_square(image_data.size)
print(f"\nLatin Square: \n{latin_square} \n\nLatin Square Shape: \n{latin_square.shape}")

# Step 2: Latin Square Whitening
whitened_data = latin_square_whitening(image_data, latin_squares)
print(f"\nWhitened Data: \n{whitened_data} \n\nWhitened Data Shape: \n{whitened_data.shape}")

# Step 3: Latin Square Substitution
substituted_data = latin_square_substitution(whitened_data.flatten(), latin_square, secret_key)
print(f"\nSubstituted Data: \n{substituted_data} \n\nSubstituted Data Shape: \n{substituted_data.shape}")

# Step 4: Latin Square Permutation
permuted_data = latin_square_permutation(substituted_data, latin_square, secret_key)
permuted_data_shape = permuted_data.shape
print(f"\nPermuted Data: \n{permuted_data} \n\nPermuted Data Shape: \n{permuted_data.shape}")

# After completing all encryption rounds, you can obtain the encrypted image
encrypted_image = permuted_data.reshape(image_data.shape)
print(f"\nEncrypted Image Shape: \n{encrypted_image.shape}")

# Display the encrypted image
encrypted_image = Image.fromarray(encrypted_image)
# encrypted_image.show()  # Show the image in a default image viewer

# Alternatively, you can save the encrypted image to a file
encrypted_image.save(os.path.join(dest, f"EncryptedImage_a{str(chr(97 + counter))}.png"))
counter += 1

# # Decryption --------------------------------------------
# src = "encrypted_images"
# dest = "decrypted_images"

# # Reverse Step 4: Latin Square Permutation
# unpermuted_data = reverse_latin_square_permutation(image_data.flatten(), latin_square, secret_key)

# # Reverse Step 3: Latin Square Substitution
# unsubstituted_data = reverse_latin_square_substitution(unpermuted_data, latin_square, secret_key)

# # Reshape data back to original dimensions
# unsubstituted_data = unsubstituted_data.reshape(image_data.shape)

# # Reverse Step 2: Latin Square Whitening
# unwhitened_data = latin_square_whitening(unsubstituted_data, latin_squares)

# decrypted_image = Image.fromarray(unwhitened_data)

# decrypted_image.save(os.path.join(dest, f"DecryptedImage_a{str(chr(97 + counter2))}.png"))
# counter2 += 1

print(f"Encrypted and decrypted {filename} successfully!")


Image shape: 
(339, 509, 3) 

Image Data: 
[[[200 202 149]
  [110 110  66]
  [ 77  74  48]
  ...
  [ 32  28  25]
  [ 63  59  55]
  [ 53  49  48]]

 [[219 221 168]
  [ 96  95  55]
  [ 91  87  68]
  ...
  [ 31  27  24]
  [ 61  57  54]
  [ 52  48  47]]

 [[225 227 172]
  [108 108  69]
  [ 79  73  68]
  ...
  [ 32  28  25]
  [ 63  59  55]
  [ 53  49  48]]

 ...

 [[ 99  97  96]
  [102 100  98]
  [103 101  99]
  ...
  [ 64  62  63]
  [ 63  61  62]
  [ 64  62  63]]

 [[103 101  99]
  [ 91  89  88]
  [ 92  90  89]
  ...
  [ 70  68  69]
  [ 70  68  69]
  [ 70  68  69]]

 [[ 91  89  88]
  [ 97  95  93]
  [102 100  98]
  ...
  [ 75  73  73]
  [ 75  73  73]
  [ 75  73  73]]]

Secret key: 
b'\xf2\xcb\x05\x06\xf0\xcb\x1d\xbb\x1a\xde\x18\xbf3u\x8c\x92/\x95\x98\x9e%\x931\x16\x17\x83\xf3\x87\xe9kS\x97'

Channels: 
3 

Latin Squares: 
[323 463 366 113 397 388 277   9 270 317 290  65 191  24 395   4 371  12
 318 196  73  75 214 497 384 423 413 476  27  28  92 295 496 183 338 107
 451 410 405 283 137 31